In [20]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt 
from matplotlib.patches import Ellipse
from sklearn.mixture import GaussianMixture
import numpy as np
import json

In [21]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [22]:
def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))

In [23]:
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covars_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

In [24]:
ram = {'MM': 'MoveMethod','PD':'PushDownMethod','PU':'PullUpMethod','EM':'ExtractMethod','IM':'InlineMethod'}
rac = {'EC':'ExtractClass','MM': 'MoveMethod','PU': 'PullUpMethod','PD':'PushDownMethod'}
files = glob("../Resources/refactoring_files/*.csv")
li = []
for filename in files:
    df = pd.read_csv(filename, header=None)
    li.append(df)
data = pd.concat(li, axis=0, ignore_index=True)

In [25]:


#TODO METHODS
dim = pd.read_csv("../Resources/csv_files/input_cluster/input_method.csv")
for item in data[0]:
    try: 
        refactoring_type = item.split("(")[0]
        item = item.split("(")[1].split(",")[0] + "()"
        key = list(ram.keys())[list(ram.values()).index(refactoring_type)]
        if (dim["Method"] == item).any():
            if key  == 'MM':
                dim.update(pd.DataFrame(
                {'Method': item, 'MM': dim.loc[dim['Method'] == item]['MM'] + 1, 'PU': dim.loc[dim['Method'] == item]['PU'],
                 'PD': dim.loc[dim['Method'] == item]['PD'], 'EM': dim.loc[dim['Method'] == item]['EM'],
                 'IM': dim.loc[dim['Method'] == item]['IM']}), ignore_index=True)
            elif key == 'PU':
                dim.update(pd.DataFrame(
                    {'Method': item, 'MM': dim.loc[dim['Method'] == item]['MM'], 'PU': dim.loc[dim['Method'] == item]['PU'] + 1,
                     'PD': dim.loc[dic['Method'] == item]['PD'], 'EM': dim.lom[dic['Method'] == item]['EM'],
                     'IM': dim.loc[dic['Method'] == item]['IM']}), ignore_index=True)
            elif key == 'PD':
                dim.update(pd.DataFrame(
                    {'Method': item, 'MM': dim.loc[dim['Method'] == item]['MM'], 'PU': dim.loc[dim['Method'] == item]['PU'],
                     'PD': dim.loc[dim['Method'] == item]['PD'] + 1, 'EM': dim.loc[dic['Method'] == item]['EM'],
                     'IM': dim.loc[dim['Method'] == item]['IM']}), ignore_index=True)
            elif key == 'EM':
                dim.update(pd.DataFrame(
                    {'Method': item, 'MM': dim.loc[dim['Method'] == item]['MM'], 'PU': dim.loc[dim['Method'] == item]['PU'],
                     'PD': dim.loc[dim['Method'] == item]['PD'], 'EM': dim.loc[dim['Method'] == item]['EM'] + 1,
                     'IM': dim.loc[dim['Method'] == item]['IM']}), ignore_index=True)
            elif key == 'IM':
                dim.update(pd.DataFrame(
                    {'Method': item, 'MM': dim.loc[dim['Method'] == item]['MM'], 'PU': dim.loc[dim['Method'] == item]['PU'],
                     'PD': dim.loc[dim['Method'] == item]['PD'], 'EM': dim.loc[dim['Method'] == item]['EM'],
                     'IM': dim.loc[dim['Method'] == item]['IM'] + 1}), ignore_index=True)
        else :
            if key  == 'MM':
                dim = dim.append({'Method':item,'MM': 1,'PU': 0,'PD': 0,'EM': 0,'IM': 0}, ignore_index=True)
            elif key  == 'PU':
                dim = dim.append({'Method':item,'MM': 0,'PU': 1,'PD': 0,'EM': 0,'IM': 0}, ignore_index=True)
            elif key  == 'PD':
                dim = dim.append({'Method':item,'MM': 0,'PU': 0,'PD': 1,'EM': 0,'IM': 0}, ignore_index=True)
            elif key  == 'EM':
                dim = dim.append({'Method':item,'MM': 0,'PU': 0,'PD': 0,'EM': 1,'IM': 0}, ignore_index=True)
            elif key  == 'IM':
                dim = dim.append({'Method':item,'MM': 0,'PU': 0,'PD': 0,'EM': 0,'IM': 1}, ignore_index=True)
    except:
        continue

dim.to_csv("../Resources/csv_files/input_cluster/input_method.csv", encoding='utf-8', index=False)
dim

,Method,MM,PU,PD,EM,IM
0,org.wordpress.android.ui.prefs.SiteSettingsFra...,0,0,0,1,0
1,org.wordpress.android.ui.main.SitePickerAdapte...,0,0,0,1,0
2,org.wordpress.android.ui.main.WPMainActivity.o...,0,0,0,1,0
3,org.wordpress.android.ui.stats.StatsUtils.open...,0,0,0,1,0
4,org.wordpress.android.models.Blog.getIconImage...,0,0,0,1,0
...,...,...,...,...,...,...
281,jetbrains.mps.project.AbstractModule.rename(),0,0,0,1,0
282,jetbrains.mps.ide.ui.tree.MPSTree.stringToPath(),0,0,0,1,0
283,jetbrains.mps.lang.editor.cellProviders.Single...,0,0,0,0,1
284,jetbrains.mps.text.impl.TextGenSupport.appendN...,0,0,0,1,0


In [26]:
dic = pd.read_csv("../Resources/csv_files/input_cluster/input_class.csv")
for item in data[0]:
    try: 
        refactoring_type = item.split("(")[0]
        key = list(rac.keys())[list(rac.values()).index(refactoring_type)]
        
        if key == "EC":
            item = item.split("(")[1].split(",")[0]
        else:
            item = item.split("(")[1].split(",")[0]
            temp = item.split(".")
            item = item.replace("."+temp[len(temp)-1], "")
        item = item.replace(")", "")
        
        if (dic["Class"] == item).any():
            if key  == 'EC':
                dic.update(pd.DataFrame({'Class': item, 'EC': dic.loc[dic['Class'] == item]['EC'] + 1,'MM': dic.loc[dic['Class'] == item]['MM'],'PU': dic.loc[dic['Class'] == item]['PU'],'PD': dic.loc[dic['Class'] == item]['PD']}))
            elif key  == 'MM':
                dic.update(pd.DataFrame({'Class': item, 'EC': dic.loc[dic['Class'] == item]['EC'],'MM': dic.loc[dic['Class'] == item]['MM']+1,'PU': dic.loc[dic['Class'] == item]['PU'],'PD': dic.loc[dic['Class'] == item]['PD']}))
            elif key  == 'PU':
                dic.update(pd.DataFrame({'Class': item, 'EC': dic.loc[dic['Class'] == item]['EC'],'MM': dic.loc[dic['Class'] == item]['MM'],'PU': dic.loc[dic['Class'] == item]['PU'] + 1,'PD': dic.loc[dic['Class'] == item]['PD']}))
            elif key  == 'PD':
                dic.update(pd.DataFrame({'Class': item, 'EC': dic.loc[dic['Class'] == item]['EC'],'MM': dic.loc[dic['Class'] == item]['MM'],'PU': dic.loc[dic['Class'] == item]['PU'],'PD': dic.loc[dic['Class'] == item]['PD']+1}))
        else :
            
            if key  == 'EC':
                dic = dic.append({'Class':item,'EC': 1,'MM': 0,'PU': 0,'PD': 0}, ignore_index=True)
            elif key  == 'MM':
                dic = dic.append({'Class':item,'EC': 0,'MM': 1,'PU': 0,'PD': 0}, ignore_index=True)
            elif key  == 'PU':
                dic = dic.append({'Class':item,'EC': 0,'MM': 0,'PU': 1,'PD': 0}, ignore_index=True)
            elif key  == 'PD':
                dic = dic.append({'Class':item,'EC': 0,'MM': 0,'PU': 0,'PD': 1}, ignore_index=True)
    except Exception as e:
#         print("ERROR : ", e)
        continue
dic.to_csv("../Resources/csv_files/input_cluster/input_class.csv", encoding='utf-8', index=False)
dic

,Class,EC,MM,PU,PD
0,io.fabric8.maven.JsonMojo,0.0,0.0,25.0,0.0
1,io.fabric8.maven.ApplyMojo,0.0,0.0,50.0,0.0
2,com.hazelcast.client.protocol.generator.CodecM...,0.0,25.0,0.0,0.0
3,com.hazelcast.jca.AbstractDeploymentTest,0.0,0.0,0.0,25.0
4,com.hazelcast.cache.impl.operation.CacheCreate...,18.0,0.0,0.0,0.0
...,...,...,...,...,...
58,tachyon.worker.block.meta.TempBlockMeta,7.0,0.0,0.0,0.0
59,org.geoserver.wfs.response.OGRWrapper,7.0,0.0,0.0,0.0
60,org.geoserver.wfs.response.OgrFormat,7.0,0.0,0.0,0.0
61,org.geoserver.wfs.response.OgrConfiguration,7.0,0.0,0.0,0.0


In [27]:
X = dic.iloc[:, 1:5]
Y = dim.iloc[:, 1:6]
d = pd.DataFrame(X)
c = pd.DataFrame(Y)

model = GaussianMixture(n_components=2, covariance_type='full', max_iter=100, n_init=1, init_params='kmeans', verbose=0, random_state=1)

# Fit the model and predict labels
clust = model.fit(d)
labels= model.predict(d)
my_class_dict = {}
for i, item in enumerate(np.unique(labels)):
    a = dic[labels == item]
    my_class_dict.update({i:[{"EC":{"MAX":int(a.EC.max()), "MIN":int(a.EC.min())}},
                         {"MM":{"MAX":int(a.MM.max()), "MIN":int(a.MM.min())}},
                          {"PU":{"MAX":int(a.PU.max()), "MIN":int(a.PU.min())}},
                          {"PD":{"MAX":int(a.PU.max()), "MIN":int(a.PD.min())}}]})

#     my_class_dict = {i:{}}

model = GaussianMixture(n_components=6, covariance_type='full', max_iter=100, n_init=1, init_params='kmeans', verbose=0, random_state=1)

# Fit the model and predict labels
clust = model.fit(c)
labels1 = model.predict(c)
my_method_dict = {}
for i, item in enumerate(np.unique(labels1)):
    a = dim[labels1 == item]
    my_method_dict.update({i:[{"MM":{"MAX":int(a.MM.max()), "MIN":int(a.MM.min())}},
                          {"PU":{"MAX":int(a.PU.max()), "MIN":int(a.PU.min())}},
                          {"PD":{"MAX":int(a.PU.max()), "MIN":int(a.PD.min())}},
                             {"EM":{"MAX":int(a.EM.max()), "MIN":int(a.EM.min())}},
                              {"IM":{"MAX":int(a.IM.max()), "MIN":int(a.IM.min())}}
                             ]})

    
with open('../Resources/json_files/class_cluster.json', 'w') as fp:
    json.dump(my_class_dict, fp)
    
with open('../Resources/json_files/method_cluster.json', 'w') as fp:
    json.dump(my_method_dict, fp)

/home/y/Desktop/iust/compiler/Final_Project/venv/lib/python3.10/site-packages/sklearn/mixture/_base.py:119: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  .fit(X)
